# Spell Checker

### Spellings mistakes

In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd

# Load the Excel file
excel_file = "data-spell-checker.xlsx"
df = pd.read_excel(excel_file, engine='openpyxl')

# Save as CSV file
csv_file = "data-spell-checker.csv"
df.to_csv(csv_file, index=False)

print(f"Excel file converted and saved as {csv_file}.")

# Load the CSV file to confirm
df= pd.read_csv(csv_file)

df.head()

Excel file converted and saved as data-spell-checker.csv.


,word,label
0,අභිචෝදකයා,1
1,අංකනය,1
2,අංකන,1
3,අංකය,1
4,අංකාන්තරය,1


In [ ]:
import re
from difflib import get_close_matches
import pandas as pd

# Load dataset with correct words
excel_file = "data-spell-checker.xlsx"
df = pd.read_excel(excel_file, engine='openpyxl')
correct_words = set(df['word'])

def tokenize_sinhala_text(paragraph):
    """Tokenize Sinhala text by matching Unicode characters."""
    sinhala_pattern = r'[\u0D80-\u0DFF]+'  # Match Sinhala Unicode range
    words = re.findall(sinhala_pattern, paragraph)
    return words

def is_vowel_modifier_difference(word1, word2):
    """Check if the difference between two words is only in vowel modifiers."""
    base_word1 = re.sub(r'[\u0DCF-\u0DFF]', '', word1)  # Remove vowel modifiers from word1
    base_word2 = re.sub(r'[\u0DCF-\u0DFF]', '', word2)  # Remove vowel modifiers from word2
    return base_word1 == base_word2

def suggest_corrections(word, correct_words, n=3):
    """Suggest the closest matching words from the dataset."""
    suggestions = get_close_matches(word, correct_words, n=n, cutoff=0.6)
    return suggestions

def correct_sinhala_text(paragraph):
    """Correct spelling mistakes in a Sinhala paragraph."""
    tokens = tokenize_sinhala_text(paragraph)
    corrected_paragraph = paragraph
    corrections = []
    corrected_words = {}  # Track already corrected words

    for word in tokens:
        if word in corrected_words:
            continue  # Skip if the word has already been corrected
        if word not in correct_words:
            suggestions = suggest_corrections(word, correct_words)
            if suggestions:
                # Check if any suggestion differs only by vowel modifiers
                for suggestion in suggestions:
                    if is_vowel_modifier_difference(word, suggestion):
                        corrected_paragraph = corrected_paragraph.replace(word, suggestion)
                        corrections.append((word, suggestion))
                        corrected_words[word] = suggestion  # Mark this word as corrected
                        break
                else:
                    # No suggestion satisfies the vowel modifier rule
                    corrections.append((word, suggestions))
                    corrected_words[word] = None  # Track as not replaceable

    return corrected_paragraph, corrections

# Example usage
input_paragraph = "මම බත් කනවා. ඔහු පාසල ට ගියේය."
corrected_paragraph, corrections = correct_sinhala_text(input_paragraph)

print("Corrected Paragraph:", corrected_paragraph)
print("Corrections:")
for original, corrected in corrections:
    if isinstance(corrected, list):
        print(f"Original: {original}, Suggestions: {corrected}")
    else:
        print(f"Original: {original}, Corrected: {corrected}")


Corrected Paragraph: මම බත් කනවා. ඔහු පාසැල ට ගියේය.
Corrections:
Original: පාසල, Corrected: පාසැල
Original: ගියේය, Suggestions: ['ගියරය', 'ගිය', 'හරියේය']


# Grammer Checker

In [ ]:
# Load the Excel file
excel_file = "verbs.xlsx"
df = pd.read_excel(excel_file, engine='openpyxl')

# Save as CSV file
csv_file = "verbs.csv"
df.to_csv(csv_file, index=False)

print(f"Excel file converted and saved as {csv_file}.")

# Load the CSV file to confirm
df= pd.read_csv(csv_file)

df.sample(5)

Excel file converted and saved as verbs.csv.


,Normal verb - Past tense,Normal verb - Present tense,Written verb - Past tense,Written verb - Present tense,Subject
84,හැදුවා,හදනවා,හැදුවෝය,හදති,ඔවුහු
81,හැදුවා,හදනවා,හැදුවෙමු,හදමු,අපි
32,කෑවා,කනවා,කෑවේය,කයි,ඔහු
2,ලිවුවා,ලියනවා,ලිව්වේය,ලියයි,ඔහු
127,සෙව්වා,සොයනවා,සෙව්වේය,සොයයි,ඔහු


### Sub model 1 to predict present tence verbs

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
data = pd.read_csv('verbs.csv')

# Extract relevant columns for the new dataset
new_data = data[['Subject', 'Normal verb - Present tense', 'Written verb - Present tense']]

# Define input (X) and target (y)
X = new_data[['Subject', 'Normal verb - Present tense']]
y = new_data['Written verb - Present tense']

# Encode the target column
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)

# Combine 'Subject' and 'Normal verb - Present tense' for feature extraction
X_combined = X['Subject'] + ' ' + X['Normal verb - Present tense']

# Split data into training and testing sets with a fixed random state
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y_encoded, test_size=0.01, shuffle=False
)

# Vectorize the input text data
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train a Random Forest Classifier with a fixed random state
model = RandomForestClassifier(random_state=42)
model.fit(X_train_vectorized, y_train)

# Make predictions
y_pred = model.predict(X_test_vectorized)

# Decode predictions back to the original labels
y_pred_decoded = target_encoder.inverse_transform(y_pred)
y_test_decoded = target_encoder.inverse_transform(y_test)

# Function to predict for new input
def predict_written_present_verb(subject, normal_verb):
    input_text = subject + ' ' + normal_verb
    input_vectorized = vectorizer.transform([input_text])
    prediction = model.predict(input_vectorized)
    return target_encoder.inverse_transform(prediction)[0]

# Example prediction
example_subject = "අපි"
example_verb = "බලනවා"
predicted_written_present_verb = predict_written_present_verb(example_subject, example_verb)
print(f"Predicted Written Verb for '{example_subject}' and '{example_verb}': {predicted_written_present_verb}")


Predicted Written Verb for 'අපි' and 'බලනවා': බලමු 


### Sub model 2 to predict past tence verbs

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
data = pd.read_csv('verbs.csv')

# Extract relevant columns for the new dataset
new_data = data[['Subject', 'Normal verb - Past tense', 'Written verb - Past tense']]

# Define input (X) and target (y)
X = new_data[['Subject', 'Normal verb - Past tense']]
y = new_data['Written verb - Past tense']

# Encode the target column
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)

# Combine 'Subject' and 'Normal verb - Past tense' for feature extraction
X_combined = X['Subject'] + ' ' + X['Normal verb - Past tense']

# Split data into training and testing sets with a fixed random state
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y_encoded, test_size=0.01, shuffle=False
)

# Vectorize the input text data
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train a Random Forest Classifier with a fixed random state
model = RandomForestClassifier(random_state=42)
model.fit(X_train_vectorized, y_train)

# Make predictions
y_pred = model.predict(X_test_vectorized)

# Decode predictions back to the original labels
y_pred_decoded = target_encoder.inverse_transform(y_pred)
y_test_decoded = target_encoder.inverse_transform(y_test)

# Function to predict for new input
def predict_written_past_verb(subject, normal_verb):
    input_text = subject + ' ' + normal_verb
    input_vectorized = vectorizer.transform([input_text])
    prediction = model.predict(input_vectorized)
    return target_encoder.inverse_transform(prediction)[0]

# Example prediction
example_subject = "ඇය"
example_verb = "ගත්තා"
predicted_written_past_verb = predict_written_past_verb(example_subject, example_verb)
print(f"Predicted Written Verb for '{example_subject}' and '{example_verb}': {predicted_written_past_verb}")


Predicted Written Verb for 'ඇය' and 'ගත්තා': ගත්තාය


### Grammer correction

In [ ]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.6 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=e931940960922cd22ba9dbdc0cafd03494199236a6e60ee881fe083e53de3f85
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Suc

In [ ]:
import pandas as pd
import re
from googletrans import Translator
import spacy

# Load the English model for spaCy
nlp = spacy.load("en_core_web_sm")

# Initialize the Google Translate API
translator = Translator()

def process_sinhala_paragraph(paragraph):
    # Step 1: Split the paragraph into sentences
    sentences = re.split(r'\.(?=\s)', paragraph.strip())  # Split by period

    # Remove full stop from the last sentence if it exists
    if sentences and sentences[-1].endswith('.'):
        sentences[-1] = sentences[-1][:-1]  # Remove the last period from the last sentence

    # List to hold the results for each sentence
    results = []

    # Variable to hold the final modified paragraph
    final_modified_paragraph = ""

    for sentence in sentences:
        # Step 2: Translate the Sinhala sentence to English
        translation = translator.translate(sentence, src='si', dest='en').text

        # Step 3: Parse the English translation using spaCy
        doc = nlp(translation)

        # Step 4: Identify subject and verb in the English sentence
        subject = None
        verb = None
        for token in doc:
            # Identify subject (subject in the dependency parse tree)
            if 'subj' in token.dep_:
                subject = token.text
            # Identify verb (verb in the dependency parse tree)
            if 'VERB' in token.pos_:
                verb = token.text

        # Step 5: Identify the tense of the sentence based on the verb
        tense = "Unknown"
        if any([token.tag_ in ['VB', 'VBP', 'VBZ'] for token in doc]):  # Present tense
            tense = "Present"
        elif any([token.tag_ in ['VBD'] for token in doc]):  # Past tense
            tense = "Past"

        # Step 6: Identify the subject and verb in the original Sinhala sentence
        sinhalawords = sentence.split()
        sinhalasubject = None
        sinhalaverb = None

        # Check the structure of the Sinhala sentence (based on the example tense rules)
        # Example approach: The first word may be the subject and last may be the verb.
        if len(sinhalawords) > 1:
            sinhalasubject = sinhalawords[0]  # First word as subject
            sinhalaverb = sinhalawords[-1]  # Last word as verb

        # Step 7: If tense is Present, replace the verb using Sub model 1
        if tense == "Present" and sinhalasubject and sinhalaverb:
            # Call Sub model 1 to predict the verb based on subject and verb
            predicted_verb = predict_written_present_verb(sinhalasubject, sinhalaverb)

            # Replace the original verb with the predicted verb
            sentence = sentence.replace(sinhalaverb, predicted_verb)

        # Step 8: If tense is Past, replace the verb using Sub model 2
        if tense == "Past" and sinhalasubject and sinhalaverb:
            # Call Sub model 1 to predict the verb based on subject and verb
            predicted_verb = predict_written_past_verb(sinhalasubject, sinhalaverb)

            # Replace the original verb with the predicted verb
            sentence = sentence.replace(sinhalaverb, predicted_verb)

        # Record the results for this sentence
        results.append({
            "Sinhala Sentence": sentence,
            "Subject (Sinhala)": sinhalasubject,
            "Verb (Sinhala)": sinhalaverb,
            "Tense": tense
        })

        # Append the sentence to the final modified paragraph
        final_modified_paragraph += sentence + ". "  # Adding a period after each sentence

    # Convert the results into a DataFrame to display as a table
    df = pd.DataFrame(results)
    return df, final_modified_paragraph.strip()


In [ ]:
# Call the function and get the results
grammar_results, final_paragraph = process_sinhala_paragraph(corrected_paragraph)

# Print the results
print("Grammar Correction Results:")
display(grammar_results)
print("\nFinal Modified Paragraph:")
final_paragraph


Grammar Correction Results:


,Sinhala Sentence,Subject (Sinhala),Verb (Sinhala),Tense
0,මම බත් දැල්වුවෙමි,මම,කනවා,Present
1,ඔහු පාසැල ට සෙව්වේය,ඔහු,ගියේය,Past



Final Modified Paragraph:


'මම බත් දැල්වුවෙමි.  ඔහු පාසැල ට සෙව්වේය.'

### Display final output

### Paragraph 1

In [ ]:
# Example paragraph for testing
input_paragraph = "අපි ඊයේ ගෙදර සිටියෙමු. ඇය පාසලේ දීප්තියෙන් බැබලෙනවා. අපි ඔන්චිල්ලා වල සෙල්ලම් කළෙමු. ඔහු දොරටුව අසල අයිස්ක්‍රීම් කනවා. ඔවුහු පොත ගත්තා"

# Step 1: Correct spelling errors
corrected_paragraph, corrections = correct_sinhala_text(input_paragraph)

# Step 2: Display corrected paragraph and suggestions
print("Original Paragraph:", input_paragraph)
print("\nCorrected Paragraph:", corrected_paragraph)
print("\nCorrections:")
for original, corrected in corrections:
    if isinstance(corrected, list):
        print(f"Original: {original}, Suggestions: {corrected}")
    else:
        print(f"Original: {original}, Corrected: {corrected}")

# Step 3: Grammar checking and further modification
# Call the function and get the results
grammar_results, final_paragraph = process_sinhala_paragraph(input_paragraph)

# Print the results
print("\nGrammar Correction Results:")
display(grammar_results)

print("\nFinal Modified Paragraph:")
final_paragraph

Original Paragraph: අපි ඊයේ ගෙදර සිටියෙමු. ඇය පාසලේ දීප්තියෙන් බැබලෙනවා. අපි ඔන්චිල්ලා වල සෙල්ලම් කළෙමු. ඔහු දොරටුව අසල අයිස්ක්‍රීම් කනවා. ඔවුහු පොත ගත්තා

Corrected Paragraph: අපි ඊයේ ගෙදර සිටියෙමු. ඇය පාසැල දීප්තියෙන් බැබලෙනවා. අපි ඔන්චිල්ලා වල සෙල්ලම් කළම. ඔහු දොරටුව අසල අයිස්ක්‍රීම් කනවා. ඔවුහු පොත ගැත්තා

Corrections:
Original: පාසලේ, Corrected: පාසැල
Original: දීප්තියෙන්, Suggestions: ['දීප්තිය', 'සමාප්තියෙන්', 'සමපත්තියෙන්']
Original: ඔන්චිල්ලා, Suggestions: ['ඔංචිල්ලාව', 'සුනිල්ලා', 'ස්කිල්ලා']
Original: කළෙමු, Corrected: කළම
Original: අයිස්ක්, Suggestions: ['අයිකන්', 'අයිකණ්', 'අයස්කාන්ත']
Original: රීම්, Suggestions: ['හැරීම්', 'හරීනම්', 'සීරීම්']
Original: ගත්තා, Corrected: ගැත්තා

Grammar Correction Results:


,Sinhala Sentence,Subject (Sinhala),Verb (Sinhala),Tense
0,අපි ඊයේ ගෙදර කිව්වෙමු,අපි,සිටියෙමු,Past
1,ඇය පාසලේ දීප්තියෙන් බැබලුනාය,ඇය,බැබලෙනවා,Present
2,අපි ඔන්චිල්ලා වල සෙල්ලම් කළෙමු,අපි,කළෙමු,Past
3,ඔහු දොරටුව අසල අයිස්ක්‍රීම් සෙව්වේය,ඔහු,කනවා,Present
4,ඔවුහු පොත ගත්තෝය,ඔවුහු,ගත්තා,Past



Final Modified Paragraph:


'අපි ඊයේ ගෙදර කිව්වෙමු.  ඇය පාසලේ දීප්තියෙන් බැබලුනාය.  අපි ඔන්චිල්ලා වල සෙල්ලම් කළෙමු.  ඔහු දොරටුව අසල අයිස්ක්\u200dරීම් සෙව්වේය.  ඔවුහු පොත\xa0ගත්තෝය           .'

### Paragraph 2

In [ ]:
# Example paragraph for testing
input_paragraph = "ඇය ඇදුම් ඇසුරුම් කළා. අපි අපේ සැලසුම් ගැන කතා කළා. මම ඉටිපන්දමක් දැල්වුවෙමි. ඇය බිස්කට් ටිකක් ගෙනාවා. ඔහු ගිටාරය වාදනය කළා."

# Step 1: Correct spelling errors
corrected_paragraph, corrections = correct_sinhala_text(input_paragraph)

# Step 2: Display corrected paragraph and suggestions
print("Original Paragraph:", input_paragraph)
print("\nCorrected Paragraph:", corrected_paragraph)
print("\nCorrections:")
for original, corrected in corrections:
    if isinstance(corrected, list):
        print(f"Original: {original}, Suggestions: {corrected}")
    else:
        print(f"Original: {original}, Corrected: {corrected}")

# Step 3: Grammar checking and further modification
# Call the function and get the results
grammar_results, final_paragraph = process_sinhala_paragraph(input_paragraph)

# Print the results
print("\nGrammar Correction Results:")
display(grammar_results)

print("\nFinal Modified Paragraph:")
final_paragraph

Original Paragraph: ඇය ඇදුම් ඇසුරුම් කළා. අපි අපේ සැලසුම් ගැන කතා කළා. මම ඉටිපන්දමක් දැල්වුවෙමි. ඇය බිස්කට් ටිකක් ගෙනාවා. ඔහු ගිටාරය වාදනය කළා.

Corrected Paragraph: ඇය ඇදුම් ඇසුරුම් කළා. අපි අපේ සැලසුම් ගැන කත කළා. මම ඉටිපන්දමක් දැල්වුවෙමි. ඇය බිස්කට් ටිකක් ගෙනාවා. ඔහු ගිටාරය වාදනය කළා.

Corrections:
Original: ඇදුම්, Suggestions: ['දුම්', 'ඇදුම', 'ඇදුම්ඛන']
Original: ඇසුරුම්, Suggestions: ['ඇසුරුම', 'සුරුවම්', 'සුරකුම්']
Original: කතා, Corrected: කත
Original: ඉටිපන්දමක්, Suggestions: ['ඉටිපන්දම', 'ඉටිපණ්දම', 'පන්දම්']
Original: දැල්වුවෙමි, Suggestions: ['සැලුවෙමි', 'සැලකුවෙමි', 'දැල්වෙන']
Original: ගෙනාවා, Suggestions: ['ගෙවනවා', 'හැගෙනවා', 'නැගෙනවා']

Grammar Correction Results:


,Sinhala Sentence,Subject (Sinhala),Verb (Sinhala),Tense
0,ඇය ඇදුම් ඇසුරුම් කළාය,ඇය,කළා,Past
1,අපි අපේ සැලසුම් ගැන කතා කළෙමු,අපි,කළා,Past
2,මම ඉටිපන්දමක් දැල්වුවෙමි,මම,දැල්වුවෙමි,Past
3,ඇය බිස්කට් ටිකක් ගියාය,ඇය,ගෙනාවා,Past
4,ඔහු ගිටාරය වාදනය කළේය,ඔහු,කළා,Past



Final Modified Paragraph:


'ඇය ඇදුම් ඇසුරුම් කළාය.  අපි අපේ සැලසුම් ගැන කතා කළෙමු.  මම ඉටිපන්දමක් දැල්වුවෙමි.  ඇය බිස්කට් ටිකක් ගියාය.  ඔහු ගිටාරය වාදනය\xa0කළේය.'

### Paragraph 3

In [ ]:
# Example paragraph for testing
input_paragraph = "ඇය ගිය සති අන්තයේ පන්සලට ගියා. ඔවුහු මහ හඬින් කතා කළා. ඔවුහු වඳුරන් ගස් උඩ පැද්දෙන හැටි බැලුවා. අපි පින්තූර ගොඩක් ගත්තා."

# Step 1: Correct spelling errors
corrected_paragraph, corrections = correct_sinhala_text(input_paragraph)

# Step 2: Display corrected paragraph and suggestions
print("Original Paragraph:", input_paragraph)
print("\nCorrected Paragraph:", corrected_paragraph)
print("\nCorrections:")
for original, corrected in corrections:
    if isinstance(corrected, list):
        print(f"Original: {original}, Suggestions: {corrected}")
    else:
        print(f"Original: {original}, Corrected: {corrected}")

# Step 3: Grammar checking and further modification
# Call the function and get the results
grammar_results, final_paragraph = process_sinhala_paragraph(input_paragraph)

# Print the results
print("\nGrammar Correction Results:")
display(grammar_results)

print("\nFinal Modified Paragraph:")
final_paragraph

Original Paragraph: ඇය ගිය සති අන්තයේ පන්සලට ගියා. ඔවුහු මහ හඬින් කතා කළා. ඔවුහු වඳුරන් ගස් උඩ පැද්දෙන හැටි බැලුවා. අපි පින්තූර ගොඩක් ගත්තා.

Corrected Paragraph: ඇය ගිය සෝති අන්තය පන්සලට ගියා. ඔවුහු මහ හඬින් කත කළා. ඔවුහු වඳුරන් ගොස් උඩ පැද්දෙන හැටි බැලුවා. අපි පින්තූර ගොඩක් ගැත්තා.

Corrections:
Original: සති, Corrected: සෝති
Original: අන්තයේ, Corrected: අන්තය
Original: පන්සලට, Suggestions: ['පන්සල', 'පින්සල', 'පැන්සල']
Original: කතා, Corrected: කත
Original: වඳුරන්, Suggestions: ['සෝවුරන්', 'සිවුරන්', 'හවුරන']
Original: ගස්, Corrected: ගොස්
Original: ගත්තා, Corrected: ගැත්තා

Grammar Correction Results:


,Sinhala Sentence,Subject (Sinhala),Verb (Sinhala),Tense
0,ඇය ගිය සති අන්තයේ පන්සලට ගියාය,ඇය,ගියා,Past
1,ඔවුහු මහ හඬින් කතා කළෝය,ඔවුහු,කළා,Past
2,ඔවුහු වඳුරන් ගස් උඩ පැද්දෙන හැටි විකුණුවෝය,ඔවුහු,බැලුවා,Past
3,අපි පින්තූර ගොඩක් ගත්තෙමු,අපි,ගත්තා,Past



Final Modified Paragraph:


'ඇය ගිය සති අන්තයේ පන්සලට ගියාය.  ඔවුහු මහ හඬින් කතා කළෝය .  ඔවුහු වඳුරන් ගස් උඩ පැද්දෙන හැටි විකුණුවෝය.  අපි පින්තූර ගොඩක් ගත්තෙමු           .'

### Paragraph 4

In [189]:
# Example paragraph for testing
input_paragraph = "ඔහු උදේ කෑම රැගෙන ආවා. අපි සපත්තු ඇසුරුම් කළා. මම පොත් සහ කෑම පෙට්ටිය සමඟ මගේ බෑගය ඇසුරුම් කළෙමි. අපි නිවසින් පිටතට ගියෙමු."

# Step 1: Correct spelling errors
corrected_paragraph, corrections = correct_sinhala_text(input_paragraph)

# Step 2: Display corrected paragraph and suggestions
print("Original Paragraph:", input_paragraph)
print("\nCorrected Paragraph:", corrected_paragraph)
print("\nCorrections:")
for original, corrected in corrections:
    if isinstance(corrected, list):
        print(f"Original: {original}, Suggestions: {corrected}")
    else:
        print(f"Original: {original}, Corrected: {corrected}")

# Step 3: Grammar checking and further modification
# Call the function and get the results
grammar_results, final_paragraph = process_sinhala_paragraph(input_paragraph)

# Print the results
print("\nGrammar Correction Results:")
display(grammar_results)

print("\nFinal Modified Paragraph:")
final_paragraph

Original Paragraph: ඔහු උදේ කෑම රැගෙන ආවා. අපි සපත්තු ඇසුරුම් කළා. මම පොත් සහ කෑම පෙට්ටිය සමඟ මගේ බෑගය ඇසුරුම් කළෙමි. අපි නිවසින් පිටතට ගියෙමු.

Corrected Paragraph: ඔහු උදේ කෑම රැගෙන ආවා. අපි සපත්තු ඇසුරුම් කළා. මම පොත් සහ කෑම පෙට්ටිය සමඟ මගේ බෑගය ඇසුරුම් කළෙමි. අපි නිවසින් පිටතට ගියෙමු.

Corrections:
Original: උදේ, Suggestions: ['හදේ', 'දේව', 'දේය']
Original: රැගෙන, Suggestions: ['හැගෙන', 'නැගෙන', 'සුරැකගෙන']
Original: ඇසුරුම්, Suggestions: ['ඇසුරුම', 'සුරුවම්', 'සුරකුම්']
Original: පොත්, Suggestions: ['පොත්ත', 'පොත', 'හොත්']
Original: කළෙමි, Suggestions: ['හළෙමි', 'සළෙමි', 'කෙමිය']
Original: නිවසින්, Suggestions: ['නිවසන', 'සියවසින්', 'විසින්']
Original: ගියෙමු, Suggestions: ['නියමු', 'ණියමු', 'ඛියමු']

Grammar Correction Results:


,Sinhala Sentence,Subject (Sinhala),Verb (Sinhala),Tense
0,ඔහු උදේ කෑම රැගෙන රැගෙන ආවේය,ඔහු,ආවා,Past
1,අපි සපත්තු ඇසුරුම් කළෙමු,අපි,කළා,Past
2,මම පොත් සහ කෑම පෙට්ටිය සමඟ මගේ බෑගය ඇසුරුම් ක...,මම,කළෙමි,Past
3,අපි නිවසින් පිටතට කිව්වෙමු,අපි,ගියෙමු,Past



Final Modified Paragraph:


'ඔහු උදේ කෑම රැගෙන රැගෙන ආවේය.  අපි සපත්තු ඇසුරුම් කළෙමු.  මම පොත් සහ කෑම පෙට්ටිය සමඟ මගේ බෑගය ඇසුරුම් කළෙමි.  අපි නිවසින් පිටතට කිව්වෙමු.'

### Paragraph 5

In [191]:
# Example paragraph for testing
input_paragraph = "ඔහු හැමදාම පාඩම් කරනවා. අපි සිය සටහන් හොඳින් කියවයි. ඔහු ඊයේ ගණිත ගැටලු දහයක් විසඳුවා. ඔහු ඇයට දුෂ්කර ප්‍රශ්න තේරුම් ගැනීමට උදව් කළේය. ඇය ඉගෙනගත් කරුණු ඇගේ මිතුරියන්ටද උගන්වයි."

# Step 1: Correct spelling errors
corrected_paragraph, corrections = correct_sinhala_text(input_paragraph)

# Step 2: Display corrected paragraph and suggestions
print("Original Paragraph:", input_paragraph)
print("\nCorrected Paragraph:", corrected_paragraph)
print("\nCorrections:")
for original, corrected in corrections:
    if isinstance(corrected, list):
        print(f"Original: {original}, Suggestions: {corrected}")
    else:
        print(f"Original: {original}, Corrected: {corrected}")

# Step 3: Grammar checking and further modification
# Call the function and get the results
grammar_results, final_paragraph = process_sinhala_paragraph(input_paragraph)

# Print the results
print("\nGrammar Correction Results:")
display(grammar_results)

print("\nFinal Modified Paragraph:")
final_paragraph

Original Paragraph: ඔහු හැමදාම පාඩම් කරනවා. අපි සිය සටහන් හොඳින් කියවයි. ඔහු ඊයේ ගණිත ගැටලු දහයක් විසඳුවා. ඔහු ඇයට දුෂ්කර ප්‍රශ්න තේරුම් ගැනීමට උදව් කළේය. ඇය ඉගෙනගත් කරුණු ඇගේ මිතුරියන්ටද උගන්වයි.

Corrected Paragraph: ඔහු හැමදාම පාඩම් කරනවා. අපි සිය සටහන් හඳින් කියවයි. ඔහු ඊයේ ගණිත ගැටලු දහයක් විසඳුවා. ඔහු ඇයට දුෂ්කර ප්‍රශ්න තේරීම් ගැනීමට උදව් කළය. ඇය ඉගෙනගත් කරුණු ඇගේ මිතුරියන්ටද උගන්වයි.

Corrections:
Original: පාඩම්, Suggestions: ['පාඩම', 'ළාඩම්', 'ලාඩම්']
Original: හොඳින්, Corrected: හඳින්
Original: කියවයි, Suggestions: ['කියවනය', 'කියවණය', 'හැකියාවයි']
Original: දහයක්, Suggestions: ['හයක්', 'හරයක්', 'හයෙක්']
Original: විසඳුවා, Suggestions: ['විසඳනවා', 'විසඳණවා', 'සිසුවා']
Original: ඇයට, Suggestions: ['යට', 'ඇය', 'ඇට']
Original: ප්, Suggestions: ['හප්', 'පළ්', 'පස්']
Original: රශ්න, Suggestions: ['වර්න', 'රත්න', 'හර්නි']
Original: තේරුම්, Corrected: තේරීම්
Original: ගැනීමට, Suggestions: ['හටගැනීමට', 'හදාගැනීමට', 'සිපගැනීමට']
Original: උදව්, Suggestions: ['උදව්ව', 'උළව්', 'උලව්']
O

,Sinhala Sentence,Subject (Sinhala),Verb (Sinhala),Tense
0,ඔහු හැමදාම පාඩම් සෙව්වේය,ඔහු,කරනවා,Present
1,අපි සිය සටහන් හොඳින් කිව්වෙමු,අපි,කියවයි,Past
2,ඔහු ඊයේ ගණිත ගැටලු දහයක් සෙව්වේය,ඔහු,විසඳුවා,Past
3,ඔහු ඇයට දුෂ්කර ප්‍රශ්න තේරුම් ගැනීමට උදව් කළේය,ඔහු,කළේය,Present
4,ඇය ඉගෙනගත් කරුණු ඇගේ මිතුරියන්ටද ගියාය,ඇය,උගන්වයි,Present



Final Modified Paragraph:


'ඔහු හැමදාම පාඩම් සෙව්වේය.  අපි සිය සටහන් හොඳින් කිව්වෙමු.  ඔහු ඊයේ ගණිත ගැටලු දහයක් සෙව්වේය.  ඔහු ඇයට දුෂ්කර ප්\u200dරශ්න තේරුම් ගැනීමට උදව් කළේය.  ඇය ඉගෙනගත් කරුණු ඇගේ මිතුරියන්ටද ගියාය.'